In [1]:
from pyspark import SparkConf, SparkContext
import pyspark
from pyspark.sql import SparkSession
# to directly solve "Python worker failed to connect back" environment problem
import findspark
findspark.init()


file_path = r"C:\Users\ee527\Desktop\result.csv"
save_path = r"C:\Users\ee527\Desktop\ELEN-E6889_Crypto_Prediction_With_Tweet\Stream_Preprocessing\dataset\clean_result.csv"

spark = SparkSession.builder.appName("Data Cleaning").getOrCreate()

df = spark.read.format("csv").option("header", "false").load(file_path)

# 打印DataFrame的结构
df.printSchema()

# 显示前几行数据
df.show()

# 选择特定的列
df.select("_c2", "_c3").show()



root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)

+--------------------+----+--------------------+--------------------+-------------------+
|                 _c0| _c1|                 _c2|                 _c3|                _c4|
+--------------------+----+--------------------+--------------------+-------------------+
|b'\xf0\x9f\xa6\x8...| 339|2023-04-19 23:57:...|b'RT @MooniumToke...|1648838200784486400|
|    b'$100 freetag.'| 542|2023-04-19 23:54:...|b'RT @MooniumToke...|1648837610306174976|
|   b'Wee Blue Ghost'| 467|2023-04-19 23:54:...|b'- \nBTC price: ...|1648837384018993156|
|      b'Vy henituse'| 774|2023-04-19 23:53:...|b'RT @MooniumToke...|1648837137457119233|
|b'jems || 10 days...| 664|2023-04-19 23:49:...|b'RT @MooniumToke...|1648836250340827136|
|b'Jessie Duval\xe...| 200|2023-04-19 23:44:...|b'RT @ComstockRoy...|1648834983589445632|
|    b'Eu

In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import col

import emoji
import preprocessor as p
import csv
import base64

def clean_word(data):
    
    return p.clean(data)


# create customized filter
clean_word_udf = udf(clean_word, StringType())

# 假设需要清理的列名为 "text"
df_clean = df.withColumn("text_clean", clean_word_udf(col("_c3")))

# 显示前几行数据
df_clean.show()

final = df_clean.collect()


# open the file in the write mode
with open(save_path, 'w', newline='',encoding='utf-8') as f:
    # create the csv writer
    writer = csv.writer(f)
    
    for row in final:
        
        # write a row to the csv file
        writer.writerow(row)




+--------------------+----+--------------------+--------------------+-------------------+--------------------+
|                 _c0| _c1|                 _c2|                 _c3|                _c4|          text_clean|
+--------------------+----+--------------------+--------------------+-------------------+--------------------+
|b'\xf0\x9f\xa6\x8...| 339|2023-04-19 23:57:...|b'RT @MooniumToke...|1648838200784486400|b'RT : Fair launc...|
|    b'$100 freetag.'| 542|2023-04-19 23:54:...|b'RT @MooniumToke...|1648837610306174976|b'RT : Fair launc...|
|   b'Wee Blue Ghost'| 467|2023-04-19 23:54:...|b'- \nBTC price: ...|1648837384018993156|b'- \nBTC price: ...|
|      b'Vy henituse'| 774|2023-04-19 23:53:...|b'RT @MooniumToke...|1648837137457119233|b'RT : Fair launc...|
|b'jems || 10 days...| 664|2023-04-19 23:49:...|b'RT @MooniumToke...|1648836250340827136|b'RT : Fair launc...|
|b'Jessie Duval\xe...| 200|2023-04-19 23:44:...|b'RT @ComstockRoy...|1648834983589445632|b'RT : CRASHES DO...|
|